In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
import tensorflow as tf
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from keras.layers import (
    Layer,
    MultiHeadAttention,
    Dense,
    LayerNormalization,
    Dropout,
    Embedding,
    Input,
    Add
)
from keras import Model, losses, Sequential, callbacks, activations, optimizers, utils
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from typing import Literal
import numpy as np



class MaskedSparseCategoricalCrossentropy(losses.Loss):
    def __init__(self, from_logits: bool = True, pad_value: int = 0, **kwargs):
        super().__init__(**kwargs)
        self.pad_value = pad_value
        self.loss = losses.SparseCategoricalCrossentropy(from_logits, reduction="none")

    def call(self, y_true: tf.Tensor, y_pred: tf.Tensor):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(y_true != self.pad_value, dtype=loss.dtype)
        loss *= mask
        loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
        return loss

class TransformerBlock(Layer):
    def __init__(self, embedding_dim: int, num_heads: int, dropout=0.1):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.dropout = dropout
        
        self.att = MultiHeadAttention(self.num_heads, self.embedding_dim)
        self.ffn = Sequential(
            [
                Dense(self.embedding_dim, activation="relu"),
                Dense(self.embedding_dim),
            ]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-5)
        self.layernorm2 = LayerNormalization(epsilon=1e-5)

        self.dropout1 = Dropout(self.dropout)
        self.dropout2 = Dropout(self.dropout)
        
    def causal_attention_mask(self, batch_size: int, n_dest: int, n_src: int, dtype: tf.DType):
        i = tf.range(n_dest)[:, tf.newaxis]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)
    
    def call(self, inputs: tf.Tensor):
        input_shape = tf.shape(inputs)
        batch_size, seq_len = input_shape[0], input_shape[1]
        causal_mask = self.causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = {}
        config.update({
            "embedding_dim": self.embedding_dim,
            "num_heads": self.num_heads,
            "dropout": self.dropout,
        })
        return config
    
class TokenAndPositionEmbedding(Layer):
    def __init__(self, sequence_length: int, vocabulary_size: int, embedding_dim: int):
        super().__init__()
        self.sequence_length = sequence_length
        self.embedding_dim = embedding_dim
        self.vocabulary_size = vocabulary_size
        
        self.token_emb = Embedding(input_dim=self.vocabulary_size, output_dim=self.embedding_dim, mask_zero=True)
        self.pos_emb = Embedding(input_dim=self.sequence_length, output_dim=self.embedding_dim)

    def call(self, x: tf.Tensor):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = {}
        config.update({
            "sequence_length": self.sequence_length,
            "embedding_dim": self.embedding_dim,
            "vocabulary_size": self.vocabulary_size,
        })
        return config


class TextGenerator(callbacks.Callback):
    def __init__(
        self,
        seed_text: str,
        next_words: int,
        max_sequence_len: int,
        tokenizer: Tokenizer,
        top_k=10,
        print_every=1,
        model=None,
        padding: Literal["pre", "post"] = "pre",
    ):
        self.seed_text = seed_text
        self.next_words = next_words
        self.max_sequence_len = max_sequence_len
        self.tokenizer = tokenizer
        if model is not None:
            self.model: Model = model
        self.print_every = print_every
        self.k = top_k
        self.padding = padding

    def sample_from(self, logits: np.ndarray) -> np.ndarray:
        indices = logits.argpartition(-self.k)[-self.k:].astype("int32")
        logits = logits[indices]
        preds = activations.softmax(tf.expand_dims(logits, 0))
        preds = np.array(preds[0]).astype("float32")
        return np.random.choice(indices, p=preds)

    def generate_text(self) -> str:
        start_tokens = self.tokenizer.texts_to_sequences([self.seed_text])[0]
        tokens_generated = []
        while len(tokens_generated) <= self.next_words:
            x = utils.pad_sequences(
                [start_tokens], maxlen=self.max_sequence_len, padding=self.padding
            )

            y = self.model.predict_on_batch(x)[0]

            idx = -1
            if self.padding == "post":
                idx = min(len(start_tokens) - 1, self.max_sequence_len - 1)

            sample_token = self.sample_from(y[idx])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)

        token_to_word = []
        for tok in tokens_generated:
            try:
                word = self.tokenizer.index_word[tok]
                token_to_word.append(word)
            except:
                token_to_word.append("")
        txt = self.seed_text + " " + " ".join(token_to_word)
        return txt

    def on_epoch_begin(self, epoch: int, logs=None):
        if (epoch + 1) % self.print_every != 0:
            return
        txt = self.generate_text()
        print(f"Epoch: {epoch}; Generated text:\n{txt}\n")

class SaveModel(tf.keras.callbacks.Callback):
    def __init__(self, path: str, **kwargs):
        super().__init__(**kwargs)
        self.path = path
    
    def on_epoch_end(self, epoch, logs=None):
        self.model.save(self.path, save_format="tf")


In [4]:
import json
import tensorflow as tf
import string
from keras import utils
import re
import warnings
from keras.preprocessing.text import tokenizer_from_json

warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=FutureWarning)
# from dataset import *
from typing import List
from glob import glob
from keras.layers import TextVectorization
from dataset import dataset_generator, read_texts


def create_model(
    max_sequence_len: int,
    total_words: int,
    embedding_dim: int = 256,
    num_heads: int = 4,
    transformer_layers: int = 1
) -> Model:
    inputs = Input(shape=(max_sequence_len,))
    x = TokenAndPositionEmbedding(max_sequence_len, total_words, embedding_dim)(inputs)
    for _ in range(transformer_layers):
        x = TransformerBlock(embedding_dim, num_heads)(x)
    outputs = Dense(total_words)(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss=losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizers.Adam(1e-3),
    )
    return model


def preprocess(content: str) -> str:
    to_left: str = r" A-Za-ząćęłńóśźż\-.,?!:;()\n"
    content = re.sub(f"[^{to_left}]+", "", content).lower()
    # content = re.sub(f"([{string.punctuation}])", r" \1", content)
    content = re.sub("\n+", " \n ", content)
    content = re.sub(" +", " ", content)
    return content


def read_files(filenames: List[str]):
    data = []
    for filename in filenames:
        with open(filename, "r", encoding="utf8") as f:
            content = f.read().lower()
            content = preprocess(content)
            data.append(content)
    return data


VOCAB_SIZE = 100_000
LEN_MAX_LIMIT = 50
LEN_MIN_LIMIT = 10
PADDING = "post"
batch_size = 128

with open("../datasets/words/books-bajki-raw-tokenizer_100000.json", "r") as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

fnames = list(glob("../texts/bajki-extend/*")) + list(glob("../texts/books-raw/*"))
data = read_files(fnames)
ds = dataset_generator(
    data,
    tokenizer,
    LEN_MIN_LIMIT,
    LEN_MAX_LIMIT,
    3,
    for_transformer=True,
    padding="post",
    batch_size=batch_size,
)


print(
    {
        "tokenizer": tokenizer.num_words,
    }
)

{'tokenizer': 100000}


In [5]:
steps_per_epoch = 1500
epochs = 25

model = create_model(LEN_MAX_LIMIT, VOCAB_SIZE)
model.summary()

model.fit(ds, verbose=1, epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=[
    TextGenerator("dawno temu czerwony kapturek poszedł do lasu i gdy szedł obok rzeki", 60, LEN_MAX_LIMIT, tokenizer, 0, padding=PADDING),
    SaveModel('../transformer_models/model_best_2.tf')
])


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 token_and_position_embeddin  (None, 50, 256)          25612800  
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 50, 256)          1184512   
 merBlock)                                                       
                                                                 
 dense_2 (Dense)             (None, 50, 100000)        25700000  
                                                                 
Total params: 52,497,312
Trainable params: 52,497,312
Non-trainable params: 0
_________________________________________________

INFO:tensorflow:Assets written to: ../transformer_models/model_best_2.tf/assets


INFO:tensorflow:Assets written to: ../transformer_models/model_best_2.tf/assets


1500/1500 [==============================] - 269s 179ms/step - loss: 4.1649
Epoch: 1; Generated text:
dawno temu czerwony kapturek poszedł do lasu i gdy szedł obok rzeki wojenny. inni niemieccy chleb z będziemy mogli sobie zauważyć, że jeden dzień cały, a po ósmej mogli gdy wtem ujął radośnie, biegnąc na oścież. wrzask tęsknił tuż przy kielichach i stary niemal w bok uniósł się, wówczas po tej dzielnicy nie gdy nisko ze swoją kulę. 
 kapłani stanął w ogrodzie i spadła na płaszcz i aż na podwórze 
 


Epoch 2/25
1500/1500 [==============================] - ETA: 0s - loss: 3.5712

INFO:tensorflow:Assets written to: ../transformer_models/model_best_2.tf/assets


...

INFO:tensorflow:Assets written to: ../transformer_models/model_best_2.tf/assets


1500/1500 [==============================] - 266s 178ms/step - loss: 2.1031
Epoch: 24; Generated text:
dawno temu czerwony kapturek poszedł do lasu i gdy szedł obok rzeki wolno, wiedziała, że już chyliło się ku zachodowi. w przejrzystym powietrzu uspokoił się i słodkim kołysał się, zmiażdży mnie wzgardą, co mnie tą 
 toż i z przyjemnością pomyślał o tym, tak wicekról indii opuszczał don a jeszcze tysiąc lat tysiąc kaczek i nieprzyjaciel ucieka do mnie do szerokich balów, żebym ją i ile nie przez ten ostatni był to nie

Epoch 25/25
1500/1500 [==============================] - ETA: 0s - loss: 2.0655

INFO:tensorflow:Assets written to: ../transformer_models/model_best_2.tf/assets


INFO:tensorflow:Assets written to: ../transformer_models/model_best_2.tf/assets


1500/1500 [==============================] - 297s 198ms/step - loss: 2.0655


In [6]:
from keras.models import load_model

model = load_model(
    "../transformer_models/model_best_2.tf",
    custom_objects={
        "TokenAndPositionEmbedding": TokenAndPositionEmbedding,
        "TransformerBlock": TransformerBlock,
        "MaskedSparseCategoricalCrossentropy": MaskedSparseCategoricalCrossentropy
    },
)

for k in [1, 2, 4, 8]:
    txt = TextGenerator(
        "dawno temu czerwony kapturek poszedł do lasu i gdy szedł obok rzeki",
        60,
        50,
        tokenizer,
        k,
        model=model,
        padding="post",
    ).generate_text()
    print(txt)
    print('---')

dawno temu czerwony kapturek poszedł do lasu i gdy szedł obok rzeki jahorlik do tego rodzaju ludzkiego, że nie wiedzieć o wiele osób z pewnością siebie, a może i nie było to samo jak najprędzej mordercę wskazywał mu się w tym sposobem w tym sposobem w tym sposobem w tej samej rzeczy samej rzeczy samej rzeczy samej rzeczy samej rzeczy samej rzeczy samej rzeczy samej rzeczy samej rzeczy samej rzeczy samej rzeczy samej
---
dawno temu czerwony kapturek poszedł do lasu i gdy szedł obok rzeki jahorlik do tego rodzaju ludzkiego, a może i tak samo, co do tego rodzaju ludzkiego, że to samo jak się z tą samą lufę. i nie było w tej samej rzeczy na 
 nie było to samo jak to samo jak to samo jak się do               
---
dawno temu czerwony kapturek poszedł do lasu i gdy szedł obok rzeki jahorlik i 
 i nie było tam i ja mam nadzieję, że się w tym sposobem opozycja do 
 i nie tylko jeden z taką samą wysokość, pan andrzej i w tej chwili gdy zaś nie było to samo jak mi się w tej karawany, że nie opowi